In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import joblib

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Load combined dataset
df = pd.read_csv('../data/fake_or_real_news.csv')

# Convert labels to binary
df['label'] = df['label'].map({'FAKE': 0, 'REAL': 1})

# Preprocessing function
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(f"[{string.punctuation}]", "", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

# Clean the text
df['cleaned_text'] = df['text'].apply(preprocess)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['cleaned_text'])
y = df['label']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Save model and vectorizer for backend
joblib.dump(model, '../backend/model/fake_news_model.pkl')
joblib.dump(vectorizer, '../backend/model/tfidf_vectorizer.pkl')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kartik\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kartik\AppData\Roaming\nltk_data...


Accuracy: 0.9859688195991091
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4710
           1       0.98      0.99      0.99      4270

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



['../backend/model/tfidf_vectorizer.pkl']